In [1]:
!pip install yfinance

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import requests

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [3]:
# Fetch historical data for SPXS
def fetchTechnicalData(_asset):
    # Fetch data for the specified asset
    hist = yf.download(_asset, start='2010-01-01') 

    # Indicator calculations as defined earlier
    def bollinger_bands(data, window=20, num_std=2):
        rolling_mean = data['Close'].rolling(window=window).mean()
        rolling_std = data['Close'].rolling(window=window).std()
        data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
        data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
        return data

    def rsi(data, periods=14, ema=True):
        close_delta = data['Close'].diff()
        up = close_delta.clip(lower=0)
        down = -1 * close_delta.clip(upper=0)
        
        if ema:
            ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
            ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        else:
            ma_up = up.rolling(window=periods, adjust=False).mean()
            ma_down = down.rolling(window=periods, adjust=False).mean()
        
        rsi = ma_up / ma_down
        data['RSI'] = 100 - (100 / (1 + rsi))
        return data

    def woodie_pivots(data):
        # Calculate Woodie's pivot points
        data['Pivot'] = (data['High'] + data['Low'] + 2 * data['Close']) / 4
        data['R1'] = 2 * data['Pivot'] - data['Low']
        data['S1'] = 2 * data['Pivot'] - data['High']
        data['R2'] = data['Pivot'] + (data['High'] - data['Low'])
        data['S2'] = data['Pivot'] - (data['High'] - data['Low'])
        data['R3'] = data['High'] + 2 * (data['Pivot'] - data['Low'])
        data['S3'] = data['Low'] - 2 * (data['High'] - data['Pivot'])
        data['R4'] = data['Pivot'] + 3 * (data['High'] - data['Low'])
        data['S4'] = data['Pivot'] - 3 * (data['High'] - data['Low'])
        return data


    # Apply each indicator function to the data
    hist = bollinger_bands(hist)
    hist = rsi(hist)
    hist = woodie_pivots(hist)
    # Repeat for other indicators as necessary...

    # Note: No explicit parallel processing applied here due to sequential dependency of calculations on data.

    # Ensure all NaN values created by indicators are handled appropriately
    hist.dropna(inplace=True)

    return hist

spxs_data = fetchTechnicalData("SPXS")
spxs_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Bollinger_High,Bollinger_Low,RSI,Pivot,R1,S1,R2,S2,R3,S3,R4,S4
Date,,,,,,,,,,,,,,,,,,
2010-02-01,23112.50,23162.50,22500.00,22525.00,21157.138672,7195,23572.863403,18085.886597,62.005138,22678.125,22856.250000,22193.750000,23340.625000,22015.625000,23518.750000,21531.250000,24665.625000,20690.625000
2010-02-02,22350.00,22600.00,21525.00,21650.00,20335.267578,8067,23651.793620,18139.456380,54.083972,21856.250,22187.500000,21112.500000,22931.250000,20781.250000,23262.500000,20037.500000,25081.250000,18631.250000
2010-02-03,21950.00,22162.50,21637.50,21987.50,20652.275391,5815,23761.286731,18214.963269,56.397745,21943.750,22250.000000,21725.000000,22468.750000,21418.750000,22775.000000,21200.000000,23518.750000,20368.750000
2010-02-04,22512.50,24050.00,22500.00,24037.50,22577.787109,13278,24236.334980,18138.665020,67.207125,23656.250,24812.500000,23262.500000,25206.250000,22106.250000,26362.500000,21712.500000,28306.250000,19006.250000
2010-02-05,23962.50,25350.00,23750.00,23862.50,22413.412109,17406,24587.939611,18193.310389,65.709548,24206.250,24662.500000,23062.500000,25806.250000,22606.250000,26262.500000,21462.500000,29006.250000,19406.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-20,10.25,10.44,10.20,10.31,10.310000,32107100,11.167095,9.755905,41.982394,10.315,10.430000,10.190001,10.555000,10.075000,10.670000,9.950001,11.034999,9.595001
2024-02-21,10.40,10.51,10.27,10.29,10.290000,22076200,11.089897,9.763103,41.642621,10.340,10.410000,10.170000,10.580000,10.100000,10.650000,9.930000,11.059999,9.620001
2024-02-22,9.87,9.94,9.60,9.66,9.660000,29778300,11.063233,9.661767,32.672642,9.715,9.829999,9.490000,10.054999,9.375001,10.169999,9.150001,10.734998,8.695002


In [4]:
def find_oversold_breakouts(spxs_data):
    # Calculate RSI and R1 for the current date
    rsi = spxs_data['RSI']
    r1 = spxs_data['R1']
    
    # Check if RSI is oversold (less than 30) and close is greater than R1
    # On TradingView, I can identify multiple points
    condition_met = (rsi < 30)  & (spxs_data['Close'] > r1) or (spxs_data['Close'] < s4)
    
    # Print out the dates when the condition is met
    dates_when_condition_met = spxs_data.index[condition_met]
    print("Dates when the condition (RSI < 30 and Close > R1) is met:")
    print(dates_when_condition_met)

# Example usage:
print_dates_when_condition_met(spxs_data)

NameError: name 'print_dates_when_condition_met' is not defined

In general, credit spreads on a 3x inversed leverged EFT have a higher than normal probability of expiring worthless, however, the breakouts identified above are nessessary to close these credit spreads as these conditions are frequently found at the begining of a bear market. In addition to managing the risk of this individual income strategy, it can also be useful in hedging a portfolio as a whole.

It is also VERY important to note, that there are very few times when this breakout last for more than 3-4 days, on the few occations when this does occur, it is often when some of the largest drops of the SPY...

# Identify Potential Bearish Credit Spreads 

In [5]:
def round_to_nearest_50_cents(value):
    """Round the value to the nearest 50 cents."""
    return np.round(value * 2, 0) / 2

def generate_credit_spreads(data):
    # Initialize a list to hold the spreads for each day
    spreads = []
    
    # Iterate through each row in the DataFrame
    for index, row in data.iterrows():
        # Initialize a dictionary for the current day's spreads
        day_spreads = {
            'Date': index,
            'R1-R2 Put Spread': None,
            'R1-R3 Put Spread': None,
            'R1-R4 Put Spread': None,
            'S1-S2 Call Spread': None,
            'S1-S3 Call Spread': None,
            'S1-S4 Call Spread': None,
        }
        
        # Calculate the put credit spreads for each pair of resistances
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['R1'])
            buy_strike = round_to_nearest_50_cents(row[f'R{i}'])
            day_spreads[f'R1-R{i} Put Spread'] = (sell_strike, buy_strike)
        
        # Calculate the call credit spreads for each pair of supports
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['S1'])
            buy_strike = round_to_nearest_50_cents(row[f'S{i}'])
            day_spreads[f'S1-S{i} Call Spread'] = (sell_strike, buy_strike)
        
        # Add the current day's spreads to the list
        spreads.append(day_spreads)
    
    # Convert the list of spreads into a DataFrame for easier viewing
    spreads_df = pd.DataFrame(spreads)
    spreads_df.set_index('Date', inplace=True)
    return spreads_df

# Assuming spxs_data is already defined and contains the necessary columns
# Generate the credit spreads
credit_spreads = generate_credit_spreads(spxs_data)

credit_spreads

,R1-R2 Put Spread,R1-R3 Put Spread,R1-R4 Put Spread,S1-S2 Call Spread,S1-S3 Call Spread,S1-S4 Call Spread
Date,,,,,,
2010-02-01,"(22856.0, 23340.5)","(22856.0, 23519.0)","(22856.0, 24665.5)","(22194.0, 22015.5)","(22194.0, 21531.0)","(22194.0, 20690.5)"
2010-02-02,"(22187.5, 22931.0)","(22187.5, 23262.5)","(22187.5, 25081.0)","(21112.5, 20781.0)","(21112.5, 20037.5)","(21112.5, 18631.0)"
2010-02-03,"(22250.0, 22469.0)","(22250.0, 22775.0)","(22250.0, 23519.0)","(21725.0, 21419.0)","(21725.0, 21200.0)","(21725.0, 20369.0)"
2010-02-04,"(24812.5, 25206.0)","(24812.5, 26362.5)","(24812.5, 28306.0)","(23262.5, 22106.0)","(23262.5, 21712.5)","(23262.5, 19006.0)"
2010-02-05,"(24662.5, 25806.0)","(24662.5, 26262.5)","(24662.5, 29006.0)","(23062.5, 22606.0)","(23062.5, 21462.5)","(23062.5, 19406.0)"
...,...,...,...,...,...,...
2024-02-20,"(10.5, 10.5)","(10.5, 10.5)","(10.5, 11.0)","(10.0, 10.0)","(10.0, 10.0)","(10.0, 9.5)"
2024-02-21,"(10.5, 10.5)","(10.5, 10.5)","(10.5, 11.0)","(10.0, 10.0)","(10.0, 10.0)","(10.0, 9.5)"
2024-02-22,"(10.0, 10.0)","(10.0, 10.0)","(10.0, 10.5)","(9.5, 9.5)","(9.5, 9.0)","(9.5, 8.5)"


In [6]:
from scipy.stats import norm

def calculate_option_probabilities(spxs_data, credit_spreads, volatility, risk_free_rate, time_to_expiration):
    """
    Calculate the probabilities of credit spreads expiring worthless using the Black-Scholes model.
    
    Parameters:
    - spxs_data: DataFrame containing 'Close' prices of the stock.
    - credit_spreads: DataFrame containing the strike prices for the spreads.
    - volatility: Annualized volatility of the stock.
    - risk_free_rate: Annual risk-free interest rate.
    - time_to_expiration: Time to expiration of the options in years.
    
    Returns:
    - DataFrame with probabilities of each spread expiring worthless.
    """
    def calculate_cdf(strike, current_price, volatility, time_to_expiration, risk_free_rate):
        """Calculate the cumulative distribution function for Black-Scholes."""
        d1 = (np.log(current_price / strike) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
        return norm.cdf(d1)
    
    probabilities = []
    
    for index, row in spxs_data.iterrows():
        current_price = row['Close']
        
        # Adjusted to directly use 'R' and 'S' values from `credit_spreads`
        for i in range(1, 5):
            if f'R{i}' in credit_spreads.columns and f'S{i}' in credit_spreads.columns:
                # For put spreads, using R values
                sell_strike_put = row[f'R{i}']
                buy_strike_put = row[f'R{i}'] - 1  # Example adjustment, customize as needed
                prob_put = calculate_cdf(sell_strike_put, current_price, volatility, time_to_expiration, risk_free_rate)
                
                # For call spreads, using S values
                sell_strike_call = row[f'S{i}']
                buy_strike_call = row[f'S{i}'] + 1  # Example adjustment, customize as needed
                prob_call = 1 - calculate_cdf(buy_strike_call, current_price, volatility, time_to_expiration, risk_free_rate)
                
                probabilities.append({
                    'Date': index,
                    f'R{i} Put Spread Probability': prob_put,
                    f'S{i} Call Spread Probability': prob_call
                })
    
    probabilities_df = pd.DataFrame(probabilities).set_index('Date')
    return probabilities_df

# Assuming you have calculated or have the values for the following variables:
volatility = 0.2  # Example volatility
risk_free_rate = 0.01  # Example risk-free rate
time_to_expiration = 1/52  # 1 week to expiration

# Calculate probabilities
probabilities_df = calculate_option_probabilities(spxs_data, spxs_data, volatility, risk_free_rate, time_to_expiration)
print(probabilities_df)

            R1 Put Spread Probability  S1 Call Spread Probability  \
Date                                                                
2010-02-01                   0.306581                    0.290015   
2010-02-01                        NaN                         NaN   
2010-02-01                        NaN                         NaN   
2010-02-01                        NaN                         NaN   
2010-02-02                   0.193956                    0.177344   
...                               ...                         ...   
2024-02-23                        NaN                         NaN   
2024-02-26                   0.391643                    0.999409   
2024-02-26                        NaN                         NaN   
2024-02-26                        NaN                         NaN   
2024-02-26                        NaN                         NaN   

            R2 Put Spread Probability  S2 Call Spread Probability  \
Date                             

In [7]:
from scipy.stats import norm

def calculate_option_probabilities(spxs_data, call_spreads, volatility, risk_free_rate, time_to_expiration):
    """
    Calculate the probabilities of bearish call spreads expiring worthless using the Black-Scholes model.
    
    Parameters:
    - spxs_data: DataFrame containing 'Close' prices of the stock.
    - call_spreads: DataFrame containing the strike prices for the call spreads.
    - volatility: Annualized volatility of the stock.
    - risk_free_rate: Annual risk-free interest rate.
    - time_to_expiration: Time to expiration of the options in years.
    
    Returns:
    - DataFrame with probabilities of each bearish call spread expiring worthless.
    """
    def calculate_cdf(strike, current_price, volatility, time_to_expiration, risk_free_rate):
        """Calculate the cumulative distribution function for Black-Scholes."""
        d1 = (np.log(current_price / strike) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
        return norm.cdf(d1)
    
    probabilities = []
    
    for index, row in spxs_data.iterrows():
        current_price = row['Close']
        
        # Adjusted to directly use 'S' values from `call_spreads`
        for i in range(1, 5):
            if f'S{i}' in call_spreads.columns:
                # For call spreads, using S values
                sell_strike_call = row[f'S{i}']
                buy_strike_call = row[f'S{i}'] + 1  # Example adjustment, customize as needed
                prob_call = 1 - calculate_cdf(buy_strike_call, current_price, volatility, time_to_expiration, risk_free_rate)
                
                probabilities.append({
                    'Date': index,
                    f'S{i} Call Spread Probability': prob_call
                })
    
    probabilities_df = pd.DataFrame(probabilities).set_index('Date')
    return probabilities_df

# Assuming you have calculated or have the values for the following variables:
volatility = 0.2  # Example volatility
risk_free_rate = 0.01  # Example risk-free rate
time_to_expiration = 1/52  # 1 week to expiration

# Calculate probabilities
probabilities_df = calculate_option_probabilities(spxs_data, spxs_data, volatility, risk_free_rate, time_to_expiration)

print(probabilities_df)


            S1 Call Spread Probability  S2 Call Spread Probability  \
Date                                                                 
2010-02-01                    0.290015                         NaN   
2010-02-01                         NaN                    0.199370   
2010-02-01                         NaN                         NaN   
2010-02-01                         NaN                         NaN   
2010-02-02                    0.177344                         NaN   
...                                ...                         ...   
2024-02-23                         NaN                         NaN   
2024-02-26                    0.999409                         NaN   
2024-02-26                         NaN                    0.998148   
2024-02-26                         NaN                         NaN   
2024-02-26                         NaN                         NaN   

            S3 Call Spread Probability  S4 Call Spread Probability  
Date                

In [8]:
def search_high_probability_spreads(probabilities_df):
    """
    Search for spreads with probabilities over 50% in the DataFrame and do not contain NaN values.
    
    Parameters:
    - probabilities_df: DataFrame containing probabilities of spreads expiring worthless.
    
    Returns:
    - DataFrame with spreads meeting the criteria.
    """
    # Filter spreads with probabilities over 50%
    high_prob_spreads = probabilities_df[(probabilities_df > 0.5).all(axis=1)]
    
    # Drop rows containing NaN values
    high_prob_spreads = high_prob_spreads.dropna()
    
    return high_prob_spreads

# Example usage:
high_prob_spreads_df = search_high_probability_spreads(probabilities_df)
print(high_prob_spreads_df)


Empty DataFrame
Columns: [S1 Call Spread Probability, S2 Call Spread Probability, S3 Call Spread Probability, S4 Call Spread Probability]
Index: []


In [9]:
R1 = probabilities_df['S1 Call Spread Probability'].dropna()
R1

Date
2010-02-01    0.290015
2010-02-02    0.177344
2010-02-03    0.325568
2010-02-04    0.114899
2010-02-05    0.105880
                ...   
2024-02-20    0.998318
2024-02-21    0.998348
2024-02-22    0.998417
2024-02-23    0.999384
2024-02-26    0.999409
Name: S1 Call Spread Probability, Length: 3541, dtype: float64

In [11]:
 probabilities_df.loc['2024-02-26']

,S1 Call Spread Probability,S2 Call Spread Probability,S3 Call Spread Probability,S4 Call Spread Probability
Date,,,,
2024-02-26,0.999409,NaN,NaN,NaN
2024-02-26,NaN,0.998148,NaN,NaN
2024-02-26,NaN,NaN,0.996494,NaN
2024-02-26,NaN,NaN,NaN,0.963241
